In [1]:
!pip install unsloth vllm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 2.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of unsloth-zoo to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of ray[cgraph] to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of unsloth-zoo to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opentelemetry-proto to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of ray[cgraph] to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of opentelemetry-proto to determine which version is compatible with other requirements. This could take a while

In [1]:
!pip install fsspec==2023.60 -- quiet

ERROR: Ignored the following yanked versions: 2022.8.0, 2022.8.1, 2025.3.1
ERROR: Could not find a version that satisfies the requirement fsspec==2023.60 (from versions: 0.1.0, 0.1.1, 0.1.2, 0.1.3, 0.1.4, 0.2.0, 0.2.1, 0.2.2, 0.2.3, 0.3.0, 0.3.1, 0.3.2, 0.3.3, 0.3.4, 0.3.5, 0.3.6, 0.4.0, 0.4.1, 0.4.2, 0.4.3, 0.4.4, 0.4.5, 0.5.1, 0.5.2, 0.6.0, 0.6.1, 0.6.2, 0.6.3, 0.7.0, 0.7.1, 0.7.2, 0.7.3, 0.7.4, 0.8.0, 0.8.1, 0.8.2, 0.8.3, 0.8.4, 0.8.5, 0.8.6, 0.8.7, 0.9.0, 2021.4.0, 2021.5.0, 2021.6.0, 2021.6.1, 2021.7.0, 2021.8.1, 2021.9.0, 2021.10.0, 2021.10.1, 2021.11.0, 2021.11.1, 2022.1.0, 2022.2.0, 2022.3.0, 2022.5.0, 2022.7.0, 2022.7.1, 2022.8.2, 2022.10.0, 2022.11.0, 2023.1.0, 2023.3.0, 2023.4.0, 2023.5.0, 2023.6.0, 2023.9.0, 2023.9.1, 2023.9.2, 2023.10.0, 2023.12.0, 2023.12.1, 2023.12.2, 2024.2.0, 2024.3.0, 2024.3.1, 2024.5.0, 2024.6.0, 2024.6.1, 2024.9.0, 2024.10.0, 2024.12.0, 2025.2.0, 2025.3.0, 2025.3.2, 2025.5.0, 2025.5.1)
ERROR: No matching distribution found for fsspec==2023.60


In [ ]:
import torch
print(torch.cuda.is_available())  # doit retourner True
print(torch.cuda.get_device_name(0))  # doit donner une carte NVIDIA

True
Tesla T4


In [ ]:
import os
os.kill(os.getpid(),9)

In [ ]:
!pip uninstall -y torch torchvision torchaudio
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [ ]:
import torch
print("CUDA available:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
else:
    print("⚠️ No GPU detected!")


CUDA available: False
⚠️ No GPU detected!


In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = torch.float16 # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/tinyllama-bnb-4bit", # "unsloth/llama-3-8b-bnb-4bit loading
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)



🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 06-18 17:40:27 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 06-18 17:40:27 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.6.2: Fast Llama patching. Transformers: 4.52.4. vLLM: 0.8.5.post1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/762M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/948 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

Lora Config

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    target_modules=['q_proj','k_proj', 'v_proj', 'o_proj'],
    lora_dropout=0, #prevent overfitting.
    bias="none",
    r=8, # rank
    lora_alpha=45,
    use_rslora=True,
    use_gradient_checkpointing = "unsloth",
)

Not an error, but Unsloth cannot patch MLP layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2025.6.2 patched 22 layers with 22 QKV layers, 22 O layers and 0 MLP layers.


Data Prep

In [ ]:
!pip install -U datasets

In [4]:
from datasets import load_dataset
dataset = load_dataset("Salesforce/wikitext", "wikitext-2-raw-v1")
train_data = dataset["train"]
eval_data = dataset["test"]
def tokenize(examples):
  return tokenizer(examples["text"], padding = "max_length", truncation=True, return_tensors="pt")

train_data = train_data.map(tokenize, batched=True)
eval_data = eval_data.map(tokenize, batched=True)
train_data.set_format("torch")
eval_data.set_format("torch")

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/733k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/6.36M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

Map:   0%|          | 0/36718 [00:00<?, ? examples/s]

Map:   0%|          | 0/4358 [00:00<?, ? examples/s]

In [ ]:
def formatting_func(example):
  return example["text"]

In [ ]:
print(type(train_data))

<class 'datasets.arrow_dataset.Dataset'>


In [5]:
import wandb
wandb.login(key="407fc3534c20abf2280dba3b41e7bda59d4524cf")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: fatima-zohra15 (fatima-zohra-grenoble-inp-uga) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported, UnslothTrainingArguments, UnslothTrainer
from transformers import DataCollatorForLanguageModeling
import triton
# Data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False) #not using mask language

# Trainer
trainer = UnslothTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_data,
    eval_dataset=eval_data,
    max_seq_length=2048,
    data_collator=data_collator,
    args=UnslothTrainingArguments(
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        gradient_accumulation_steps=4,
        do_eval=True,
        eval_steps=10,
        warmup_ratio=0.1,
        num_train_epochs=3,
        learning_rate=1e-4,
        embedding_learning_rate=5e-6,
        fp16 = True, # Use 16-bit floating point if bfloat16 isn't supported.
        bf16 = False,     #use bf16 if hardware support
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.00,
        lr_scheduler_type="cosine", #Cosine scheduler to adjust the learning rate over time.

        output_dir="./unsloth/lora_drop_grad",
    ),
)

# Train
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 36,718 | Num Epochs = 3 | Total steps = 6,885
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 2,252,800/617,859,072 (0.36% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,3.907500
2,4.120300
3,3.563500
4,3.581600
5,4.119900
6,4.272100
7,4.109400
8,4.619500
9,3.843500
10,4.353600


In [ ]:
!pip install -U triton torch

  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.6.77-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.6.77-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.6.80-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.5.1.17-py3-none-manylinux_2_28_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.6.4.1-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.3.0.4-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.7.77-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.7.1.2-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu

In [ ]:
print(torch.__version__)
print(triton.__version__)

2.7.1+cu126
3.3.1


In [ ]:
import os
os.environ["TRITON_PTXAS_PATH"] = "/usr/bin/false"

In [ ]:
print(torch.cuda.get_device_capability(0))  # Compute capability (doit être >= 8.0 pour BF16)

(7, 5)


In [ ]:
import torch
print("Hello")

Hello


In [ ]:
compute_cap = torch.cuda.get_device_capability()
print("🧠 GPU compute capability:", compute_cap)
assert compute_cap >= (7, 5), "❌ GPU compute capability must be >= 7.5 (T4 or higher)."

if compute_cap < (8, 0):
    print("⚠️ GPU does NOT support bf16. Forcing FP16 mode only.")
    os.environ["TORCH_ALLOW_TF32_CUBLAS"] = "0"
    os.environ["TORCH_ENABLE_BF16"] = "0"
    os.environ["XFORMERS_FORCE_DISABLE_BF16"] = "1"
    os.environ["TRITON_PTXAS_PATH"] = "/usr/bin/false"  # Optional but helps avoid ptxas calls
